- Antes de aplicar código solo he cambiado el nombre de las cabeceras para unificarlos
- He unido los csvs de life expectancy de Our World in data añadiendo la columna gender
- en la extracion analie algunos ficheros que descarté al estar incompletos oo no servir pa na. He convertido los csvs a dfs para las transformaciones
- Mergeo en 7BirthAttendedBySkilledPersonal los datos de la WHO(dado en Kaggle) y World Bank para ampliar. 151 valores no coinciden asi que hago su media
- He descargado 16Cardiovascular de la WHO para que fuera más completo y le he cambiado el formato para que encaje con el resto
- Otras transformaciones: Separar genero en Life expectancy y eliminar both sexes en neonatal
- En 26AirPollution separamos en varios csvs en función de la disease y en si es o no age -standarized
- En donde hay Area(36,37,38,24,25), separamos en varios csvs: Total, rural, urban
- Cambiar el nombre de las cabeceras de los csv para que queden unificados
- Explicar conjunto de datos de partida y el formato (csv) en que estaban, asignandole un número a cada uno
- He filtrado los datasets menos relevantes
- He hecho un git para importarlos a jupyter
- He tratado los datos para definir correctamente un DataFrame para cada csv separando algunos en varios DFs, eliminando columnas innecesarias, separando aquellas columnas que dan un intervalo de confianza para el valor cambiando los tipos de datos y renombrando las columnas.

In [1]:
# Miguel Roca García

**Unificación de varios CSVs a uno único**

# Importar librerías

In [2]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None #Para evitar warnings
import urllib.request
import re
import ast
pd.set_option("display.max_columns", None)#Para mostrar todas las columnas

# Importar datos

In [3]:
#Importamos de GitHub un JSON con los nombres de todos los CSV con los datos y sus futuros nombres de columna en el DataFrame
namesIndicatorJson = urllib.request.urlopen('https://raw.githubusercontent.com/MiguelRocaGarcia/DatasetTFG/master/IndicatorsDataFrameNames.json')
namesJson = namesIndicatorJson.read().decode()
indicatorsNames = ast.literal_eval(namesJson)

In [10]:
#Importamos los CSVs
dfs = {}
for name in indicatorsNames.keys():
    dfs[name] = pd.read_csv(f'https://raw.githubusercontent.com/MiguelRocaGarcia/DatasetTFG/master/{name}.csv', sep=',')

In [11]:
dfs['48_TotalPopulation']

,Indicator,Country,Year,Value
0,"Population, total",Afghanistan,1990.0,12412308
1,"Population, total",Afghanistan,1991.0,13299017
2,"Population, total",Afghanistan,1992.0,14485546
3,"Population, total",Afghanistan,1993.0,15816603
4,"Population, total",Afghanistan,1994.0,17075727
...,...,...,...,...
7920,NaN,NaN,NaN,NaN
7921,NaN,NaN,NaN,NaN
7922,NaN,NaN,NaN,NaN
7923,Data from database: World Development Indicators,NaN,NaN,NaN


# Transformaciones

In [5]:
# Eliminamos la columna Indicator
for df in dfs.values():
    if 'Indicator' in df.columns:
        df.drop('Indicator', axis=1, inplace=True)   

In [6]:
#Separamos los valores de las columnas estimados con su intervalo de confianza
for dfName in dfs.keys():
    if '[' in str(dfs[dfName]['Value'][0]):
        dfs[dfName]['Low Value'] = dfs[dfName]['Value'].apply(lambda x: re.search('\[(.*?)-',x).group(1))
        dfs[dfName]['High Value'] = dfs[dfName]['Value'].apply(lambda x: re.search('-(.*?)\]',x).group(1))
        dfs[dfName]['Value'] = dfs[dfName]['Value'].apply(lambda x: re.search('(.*?)\[',x).group(1))

In [7]:
#Número de filas antes de la transformación
shapes = {}
for name, df in sorted(dfs.items()):
    shapes[name] = df.shape[0]

In [8]:
#Cambiar tipos de datos: de str a int o float y eliminar los NaN
for dfName in dfs.keys():
    dfs[dfName].dropna(inplace=True)  #Eliminamos los valores NaN(Necesario para que no convierta Year a float)
    dfs[dfName]['Year'] = pd.to_numeric(dfs[dfName]['Year'], errors='coerce',downcast='integer') #Convertimos Year a integer
    dfs[dfName]['Value'] = pd.to_numeric(dfs[dfName]['Value'], errors='coerce') #Convertimos value a numérico
    if ('Low Value' in dfs[dfName].columns):
        dfs[dfName]['Low Value'] = pd.to_numeric(dfs[dfName]['Low Value'], errors='coerce')
        dfs[dfName]['High Value'] = pd.to_numeric(dfs[dfName]['High Value'], errors='coerce')
    dfs[dfName].dropna(inplace=True)  #Eliminamos los valores NaN

1_LifeExpectancyAtBirth
6_MaternalMortalityRatio
7_BirthAttendedBySkilledPersonal
8_InfantMortalityRate
9_NeonatalMortalityRate
10_Under5MortalityRate
11_IncedenceOfMalaria
12_IncedenceOfTuberculosis
13_HepatitusBsurfaceAntigen
14_InterventionAgainstNTDs
16_CardiovascularDyingProbability30_70
17_SuicideRates
18_AlcoholSubstanceAbuse
19_RoadTrafficDeaths
20_ReproductiveAgeWomen
21_AdolescentBirthRate
22_UniversalHealthCareCoverage
24-1_Population10PercentageSDGTotal
24-2_Population10PercentageSDGUrban
24-3_Population10PercentageSDGRural
25-1_Population25PercentageSDGTotal
25-2_Population25PercentageSDGUrban
25-3_Population25PercentageSDGRural
26-1-1_AirPollutionDeathRateStroke
26-1-2_AirPollutionDeathRateStrokeAgeStandardized
26-2-1_AirPollutionDeathRateIschaemicHeartDisease
26-2-2_AirPollutionDeathRateIschaemicHeartDiseaseAgeStandardized
26-3-1_AirPollutionDeathRateLowerRespiratoryInfections
26-3-2_AirPollutionDeathRateLowerRespiratoryInfectionsAgeStandardized
26-4-1_AirPollutionDeathR

KeyError: 'Year'

In [ ]:
# Número de filas tras la transformación
shapesAfter = {}
for name, df in sorted(dfs.items()):
    shapesAfter[name] = df.shape[0]

In [ ]:
#Filas eliminadas por contener NaN
for name in sorted(shapes.keys()):
    print(name, ': ', shapes[name] - shapesAfter[name])

In [ ]:
#Vemos los años disponibles para cada indicador
for name, df in sorted(dfs.items()):
  if 'Year' in df.columns:
    years = df['Year'].unique().tolist()
    print(name, '\n', years)
    print()

In [ ]:
#Filtramos por años entre 1900 y 2019
for dfName in dfs.keys():
    dfs[dfName] = dfs[dfName][(dfs[dfName]['Year'] >= 1990)]

In [ ]:
# Cambiamos los nombres de las columnas para hacerlos más significativos
for dfName in dfs.keys():
    dfs[dfName].rename(columns={'Value': indicatorsNames[dfName]}, inplace=True)
    if('Low Value' in dfs[dfName].columns):
        dfs[dfName].rename(columns={'Low Value': f'Low CI Value {indicatorsNames[dfName]}',
                                   'High Value': f'High CI Value {indicatorsNames[dfName]}'}, inplace=True)

Para añadir un nuevo CSV, solo hay que subirlo al GitHub y añadirlo en IndicatorsDataFrameNames.json
Tiene que tener las mismas cabeceras y formato que el resto.

## Unificar Conjunto de Datos

In [ ]:
# Separamos en dos listas con los nombres de los DFs que tienen y no tienen Gender para después hacer los merge
dfsWithGender = []
dfsWithoutGender = []
for dfName, df in dfs.items():
    if('Gender' in df.columns):
        dfsWithGender.append(dfName)
    else:
        dfsWithoutGender.append(dfName)
print('With gender: ', dfsWithGender, '\n\nWithout Gender: ', dfsWithoutGender)

In [ ]:
#Mergeamos por gender
df_mergedByGender = dfs[dfsWithGender[0]]
for dfName in dfsWithGender[1:]:
    df_mergedByGender = pd.merge(df_mergedByGender, dfs[dfName], 
                                 on=['Country', 'Year', 'Gender'], how='outer').sort_values(['Country', 'Year', 'Gender'])

In [ ]:
# Mergeamos los que no tienen gender
df_mergedWithoutGender = dfs[dfsWithoutGender[0]]
for dfName in dfsWithoutGender[1:]:
    df_mergedWithoutGender = pd.merge(df_mergedWithoutGender, dfs[dfName], 
                                      on=['Country', 'Year'], how='outer').sort_values(['Country', 'Year'])

In [ ]:
df = pd.merge(dfs['6_MaternalMortalityRatio'], dfs['7_BirthAttendedBySkilledPersonal'], on=['Country', 'Year'], how='outer')

In [ ]:
# Mergemos ambos merges
df_final = pd.merge(df_mergedByGender, df_mergedWithoutGender, 
                                      on=['Country', 'Year'], how='outer').sort_values(['Country', 'Year', 'Gender'])


In [ ]:
df_final.head(50)

In [ ]:
#Eliminamos aquellas filas cuyo valor Life Expectancy es NaN 
df_final.dropna(subset=['Life Expectancy'], inplace=True)
df_final.reset_index(drop=True, inplace=True)

In [ ]:
df_final

## Exportar a un CSV

In [ ]:
df_final.to_csv('UnifiedDataset.csv',sep=',', index=False)